In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        acc = accuracy(out, labels) # Calculate accuracy
        return {'train_loss': loss.detach(), 'train_acc': acc}
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [20]:
class LeNet(ImageClassificationBase):
  def __init__(self, input_channels = 1):
    super(LeNet, self).__init__()
    self.block_1 = nn.Sequential(
      nn.Conv2d(input_channels, 6, kernel_size = 5, stride = 1, padding = 0, bias = False),
      nn.BatchNorm2d(6),
      nn.Tanh(),
      nn.AvgPool2d(kernel_size = 2, stride = 2),
      nn.Conv2d(6, 16, kernel_size = 5, stride = 1, padding = 0, bias = False),
      nn.BatchNorm2d(16),
      nn.Tanh(),
      nn.AvgPool2d(kernel_size = 2, stride = 2),
      nn.Conv2d(16, 120, kernel_size = 5, stride = 1, padding = 0, bias = False),
      nn.BatchNorm2d(120),
      nn.Tanh(),
    )
    self.linear_1 = nn.Linear(120, 84)
    self.linear_2 = nn.Linear(84, 10)
    self.softmax = nn.Softmax(dim = 1)

  def forward(self, x):
    x = self.block_1(x)
    x = x.view(x.size(0), -1)
    x = self.linear_1(x)
    x = self.linear_2(x)
    x = self.softmax(x)
    return x

In [21]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision.transforms import Resize
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
from torchvision.transforms import ToTensor, Normalize, Compose
from torch.utils.data.dataloader import DataLoader

batch_size=128

train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.Compose(
        [
        Resize(size = (32,32)),
        ToTensor(),
        ]
    )
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.Compose(
        [
        Resize(size = (32,32)),
        ToTensor(),
        ]
    )
)

train_dl = DataLoader(train_data, batch_size, shuffle=True)
val_dl = DataLoader(test_data, batch_size*2)
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [22]:
img, label = train_data[0]

In [23]:
img.size()

torch.Size([1, 32, 32])

In [24]:
model = LeNet()
from torchvision import models
from torchsummary import summary

print(summary(model,(1, 32, 32)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             150
       BatchNorm2d-2            [-1, 6, 28, 28]              12
              Tanh-3            [-1, 6, 28, 28]               0
         AvgPool2d-4            [-1, 6, 14, 14]               0
            Conv2d-5           [-1, 16, 10, 10]           2,400
       BatchNorm2d-6           [-1, 16, 10, 10]              32
              Tanh-7           [-1, 16, 10, 10]               0
         AvgPool2d-8             [-1, 16, 5, 5]               0
            Conv2d-9            [-1, 120, 1, 1]          48,000
      BatchNorm2d-10            [-1, 120, 1, 1]             240
             Tanh-11            [-1, 120, 1, 1]               0
           Linear-12                   [-1, 84]          10,164
           Linear-13                   [-1, 10]             850
          Softmax-14                   

In [25]:
model = LeNet()

num_epochs = 50
opt_func = torch.optim.Adam
lr = 0.001

fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

Epoch [0], train_loss: 1.6715, val_loss: 1.6270, val_acc: 0.8367
Epoch [1], train_loss: 1.6107, val_loss: 1.6104, val_acc: 0.8519
Epoch [2], train_loss: 1.5949, val_loss: 1.6010, val_acc: 0.8610
Epoch [3], train_loss: 1.5834, val_loss: 1.5934, val_acc: 0.8687
Epoch [4], train_loss: 1.5769, val_loss: 1.5962, val_acc: 0.8648
Epoch [5], train_loss: 1.5720, val_loss: 1.5914, val_acc: 0.8696
Epoch [6], train_loss: 1.5678, val_loss: 1.5845, val_acc: 0.8798
Epoch [7], train_loss: 1.5640, val_loss: 1.5929, val_acc: 0.8676
Epoch [8], train_loss: 1.5614, val_loss: 1.5761, val_acc: 0.8848
Epoch [9], train_loss: 1.5574, val_loss: 1.5787, val_acc: 0.8828
Epoch [10], train_loss: 1.5551, val_loss: 1.5766, val_acc: 0.8840
Epoch [11], train_loss: 1.5535, val_loss: 1.5664, val_acc: 0.8938
Epoch [12], train_loss: 1.5511, val_loss: 1.5723, val_acc: 0.8889
Epoch [13], train_loss: 1.5500, val_loss: 1.5714, val_acc: 0.8894
Epoch [14], train_loss: 1.5480, val_loss: 1.5735, val_acc: 0.8874
Epoch [15], train_lo

[{'val_loss': 1.6269861459732056,
  'val_acc': 0.836718738079071,
  'train_loss': 1.671455979347229},
 {'val_loss': 1.610378623008728,
  'val_acc': 0.851855456829071,
  'train_loss': 1.6106550693511963},
 {'val_loss': 1.6009819507598877,
  'val_acc': 0.861035168170929,
  'train_loss': 1.594947099685669},
 {'val_loss': 1.59340500831604,
  'val_acc': 0.8687499761581421,
  'train_loss': 1.5833853483200073},
 {'val_loss': 1.596183180809021,
  'val_acc': 0.8648437261581421,
  'train_loss': 1.5769293308258057},
 {'val_loss': 1.5913918018341064,
  'val_acc': 0.86962890625,
  'train_loss': 1.57204008102417},
 {'val_loss': 1.5844814777374268,
  'val_acc': 0.8797851800918579,
  'train_loss': 1.5677613019943237},
 {'val_loss': 1.5929248332977295,
  'val_acc': 0.8675781488418579,
  'train_loss': 1.5640380382537842},
 {'val_loss': 1.5760712623596191,
  'val_acc': 0.884765625,
  'train_loss': 1.5614471435546875},
 {'val_loss': 1.5787490606307983,
  'val_acc': 0.8828125,
  'train_loss': 1.55735909938